In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
import requests

In [2]:
densidad = pd.read_csv('../Data/clean_data/densidad_poblacion.csv')

In [3]:
densidad.head(1)

,prov_id,mun_id,total_2023,hombres_2023,mujeres_2023,ine_id,municipio,km2,densidad
0,1,1001,2975.0,1539.0,1436.0,1001000000,Alegria-dulantzi,1995.0,1.491228


In [4]:
contagio_mun = densidad.loc[:,['densidad', 'prov_id', 'mun_id', 'municipio']]

In [5]:
contagio_mun.head(2)

,densidad,prov_id,mun_id,municipio
0,1.491228,1,1001,Alegria-dulantzi
1,1.070368,1,1002,Amurrio


In [6]:
#calculating zombie virus estimated basic reproduction number (transmision rate): infectiousnes * population density

c = 5 #constant representing infectiousness of the virus (For reference, flu is 1.3, measles is 12-18. We are picking 8 given that every human is susceptible of contracting the virus, there is no recovery and once bitten you are infected with a 100% probability.)

In [7]:
contagio_mun['R0'] = contagio_mun.densidad*c

In [8]:
contagio_mun['prob_inf'] = 1 - np.exp((-1*contagio_mun.R0*contagio_mun.densidad))

In [9]:
contagio_mun.head(3)

,densidad,prov_id,mun_id,municipio,R0,prob_inf
0,1.491228,1,1001,Alegria-dulantzi,7.456140,0.999985
1,1.070368,1,1002,Amurrio,5.351842,0.996748
2,0.192302,1,1003,Aramaio,0.961512,0.168814


In [10]:
#contagio_mun.to_csv('contagio_municipios.csv', encoding='utf-8-sig', index = False)

In [11]:
contagio_prov = pd.read_csv('../Data/clean_data/cod_provincias.csv')

In [12]:
contagio_prov.head(2)

,prov_id,com_id,provincia
0,4,1,Almería
1,11,1,Cádiz


In [13]:
#calculamos los datos de superficie, población total y densidad por provincia a partir de los datos por municipio

In [14]:
contagio_prov['km2'] = contagio_prov.prov_id.apply(lambda x: sum(densidad[densidad.prov_id == x]['km2']))

In [15]:
contagio_prov['total_prov'] = contagio_prov.prov_id.apply(lambda x: sum(densidad[densidad.prov_id == x]['total_2023']))

In [16]:
contagio_prov['densidad'] = contagio_prov['total_prov'] / contagio_prov['km2']
contagio_prov.head(2)

,prov_id,com_id,provincia,km2,total_prov,densidad
0,4,1,Almería,832312.00,753920.0,0.905814
1,11,1,Cádiz,612140.92,1250539.0,2.042894


In [17]:
#calculamos el índice y probablilidad de contagio por provincia

In [18]:
contagio_prov['R0'] = contagio_prov.densidad*c
contagio_prov['prob_inf'] = 1 - np.exp((-1*contagio_prov.R0*contagio_prov.densidad))

In [19]:
contagio_prov.head(3)

,prov_id,com_id,provincia,km2,total_prov,densidad,R0,prob_inf
0,4,1,Almería,832312.00,753920.0,0.905814,4.529071,0.983469
1,11,1,Cádiz,612140.92,1250539.0,2.042894,10.214470,1.000000
2,14,1,Córdoba,1131334.25,773997.0,0.684145,3.420726,0.903699


In [20]:
#repetimos el proceso por comunidad autónoma:

In [21]:
coms = contagio_prov.iloc[:,[0,1,3,4]]

In [22]:
coms.head(1)

,prov_id,com_id,km2,total_prov
0,4,1,832312.0,753920.0


In [23]:
contagio_com = pd.read_csv('../Data/clean_data/cod_comundades.csv')

In [24]:
contagio_com.head(1)

,com_id,comunidad
0,1,Andalucía


In [25]:
#población, superficie y densidad de pobalción por comunidad a partir de datos de provincia:

In [26]:
contagio_prov['km2'] = contagio_prov.prov_id.apply(lambda x: sum(densidad[densidad.prov_id == x]['km2']))

In [27]:
contagio_com['km2'] = contagio_com.com_id.apply(lambda x: sum(contagio_prov[contagio_prov.com_id == x]['km2']))

In [28]:
contagio_com['total_com'] = contagio_com.com_id.apply(lambda x: sum(contagio_prov[contagio_prov.com_id == x]['total_prov']))

In [29]:
contagio_com['densidad'] = contagio_com['total_com'] / contagio_com['km2']
contagio_com.head(2)

,com_id,comunidad,km2,total_com,densidad
0,1,Andalucía,7763487.17,8570528.0,1.103953
1,2,Aragón,4407470.00,1339759.0,0.303975


In [30]:
contagio_com.head(1)

,com_id,comunidad,km2,total_com,densidad
0,1,Andalucía,7763487.17,8570528.0,1.103953


In [31]:
#índice y probabilidad de contagio por comunidad:

In [32]:
contagio_com['R0'] = contagio_com.densidad*c
contagio_com['prob_inf'] = 1 - np.exp((-1*contagio_com.R0*contagio_com.densidad))

In [33]:
contagio_com.head(1)

,com_id,comunidad,km2,total_com,densidad,R0,prob_inf
0,1,Andalucía,7763487.17,8570528.0,1.103953,5.519767,0.997743


In [34]:
#contagio_com.to_csv('contagio_comunidades.csv', encoding='utf-8-sig', index = False)

In [35]:
#quitamos la columna de com_id de la tabla de contagio por provincia:

In [36]:
#contagio_prov.drop(columns=['com_id'], inplace = True)

In [37]:
contagio_prov.head(2)

,prov_id,com_id,provincia,km2,total_prov,densidad,R0,prob_inf
0,4,1,Almería,832312.00,753920.0,0.905814,4.529071,0.983469
1,11,1,Cádiz,612140.92,1250539.0,2.042894,10.214470,1.000000


In [38]:
#contagio_prov.to_csv('contagio_provincias.csv', encoding='utf-8-sig', index = False)